# Upload lifelogs and families to database
Natalia Vélez, November 2020

In [38]:
import pymongo
import os
import json
import pandas as pd
import numpy as np
from os.path import join as opj
from tqdm import notebook

Connect to database:

In [2]:
keyfile = 'credentials.key'
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
db = myclient.ohol

print(db)
print(db.list_collection_names())

Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), 'ohol')
['test']


## Lifelogs

### Prepare data

Load lifelog data:

In [3]:
log_df = pd.read_csv('../2_demographics/outputs/all_lifelogs_compact.tsv', sep='\t', index_col=0)
log_df['death'] = np.where(log_df['death'].str.len() <= 2, np.nan, log_df['death'])
print(log_df.shape)
log_df.head()

/Users/nvelez/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1826661, 15)


,player,avatar,tBirth,parent,birth,tDeath,death,age,cause_of_death,birthX,birthY,deathX,deathY,first,last
0,06d8ab86b0daf11b13a446b87f770aa20bee1ca0,3308603,1599462917,3308581,[-63752 -504],NaN,NaN,NaN,NaN,-63752,-504,NaN,NaN,SUPERIOR,MATSU
1,67bffa53500e2c626edc407409b6550996b748db,3308602,1599462864,3308575,[-63753 -507],NaN,NaN,NaN,NaN,-63753,-507,NaN,NaN,FEATHER,MATSU
2,fbc0168048713cfaf6ef6a350c0171e7b2b0af4a,3308601,1599462805,3308564,[-63762 -523],NaN,NaN,NaN,NaN,-63762,-523,NaN,NaN,SASHA,MATSU
3,5d5050247e485d0224df4e392118c27b8ee3a88b,3308598,1599462667,3308575,[-63747 -498],NaN,NaN,NaN,NaN,-63747,-498,NaN,NaN,DRAGON,MATSU
4,310d12780dbb316a3b50f83adbf9ce76973172bb,3308597,1599462620,3308571,[-63756 -508],1.599464e+09,[-63841 -520],28.89,hunger,-63756,-508,-63841.0,-520.0,MODOU,MATSU


Load expertise:

In [9]:
expert_df = pd.read_csv('../2_demographics/outputs/player_expertise.tsv', sep='\t')
expert_df = expert_df[['avatar', 'player', 'n_life', 'gametime']]
print(expert_df.shape)
expert_df.head()

(3151906, 4)


,avatar,player,n_life,gametime
0,630641,00006debdd4d630a4a5aa395135a70a69e0db01e,0,0.00
1,630737,00006debdd4d630a4a5aa395135a70a69e0db01e,1,13.88
2,630799,00006debdd4d630a4a5aa395135a70a69e0db01e,2,18.86
3,214206,0001394319e5e63bb74219fef647e9ff24507a21,0,0.00
4,214530,0001394319e5e63bb74219fef647e9ff24507a21,1,33.78


Load families:

In [5]:
fam_success = pd.read_csv('../2_demographics/outputs/family_fitness.tsv', sep='\t')
fam_members = pd.read_csv('../2_demographics/outputs/family_generations.tsv', sep='\t')
fam_success.head()

,family,sum,count,a,b,beta_mean,beta_var,snr,weighted_size,longest_path
0,time-1573253679_eve-2251043_name-MCCRACKEN,2,9,5,10,0.333333,0.013889,24.000000,3.000000,3
1,time-1573253705_eve-2251049_name-FINCH,1,8,4,10,0.285714,0.013605,21.000000,2.285714,2
2,time-1573253707_eve-2251051_name-SORROWS,1,3,4,5,0.444444,0.024691,18.000000,1.333333,2
3,time-1573253710_eve-2251053_name-LION,9,40,12,34,0.260870,0.004102,63.588235,10.434783,4
4,time-1573253711_eve-2251054_name-DEADRICK,33,86,36,56,0.391304,0.002561,152.785714,33.652174,7


Merge:

In [11]:
life_df = pd.merge(log_df, expert_df, on=['player', 'avatar'])
print(life_df.shape)
life_df = pd.merge(life_df, fam_members, on=['avatar'])
print(life_df.shape)
life_df.head()

(1825025, 17)
(470361, 19)


,player,avatar,tBirth,parent,birth,tDeath,death,age,cause_of_death,birthX,birthY,deathX,deathY,first,last,n_life,gametime,family,gen
0,310d12780dbb316a3b50f83adbf9ce76973172bb,3308597,1599462620,3308571,[-63756 -508],1.599464e+09,[-63841 -520],28.89,hunger,-63756,-508,-63841.0,-520.0,MODOU,MATSU,0,0.00,time-1599447278_eve-3308179_name-MATSU,9
1,de586e48a15f97f09e71355fcc82cea88d29633a,3308595,1599462571,3308576,[-63788 148],1.599465e+09,[-63794 187],36.92,hunger,-63788,148,-63794.0,187.0,VIOLET,SUN,36,896.57,time-1599411200_eve-3307135_name-SUN,35
2,3da17539b16d6fe6911aefc57388bae9d1303e22,3308591,1599462365,3308571,[-63753 -509],1.599465e+09,[-63761 -512],45.82,hunger,-63753,-509,-63761.0,-512.0,RAIN,MATSU,599,8587.95,time-1599447278_eve-3308179_name-MATSU,9
3,043719a2996d748e294d308706447d2e6ce30181,3308583,1599462113,3308537,[-63797 -173],1.599463e+09,[-63788 -209],20.11,hunger,-63797,-173,-63788.0,-209.0,STAR,LUNA,0,0.00,time-1599454213_eve-3308371_name-LUNA,5
4,49bd234c63fac4073c98cc23dab916528f87e634,3308581,1599461853,3308545,[-63752 -512],1.599465e+09,[-63758 -491],45.37,hunger,-63752,-512,-63758.0,-491.0,JANE,MATSU,3541,40795.99,time-1599447278_eve-3308179_name-MATSU,9


### Lifelog collection

Convert to record:

In [12]:
life_records = life_df.to_dict('records')
print('Saving %i records' % len(life_records))
print(life_records[:5])

Saving 470361 records
[{'player': '310d12780dbb316a3b50f83adbf9ce76973172bb', 'avatar': 3308597, 'tBirth': 1599462620, 'parent': 3308571, 'birth': '[-63756   -508]', 'tDeath': 1599464354.0, 'death': '[-63841   -520]', 'age': 28.89, 'cause_of_death': 'hunger', 'birthX': -63756, 'birthY': -508, 'deathX': -63841.0, 'deathY': -520.0, 'first': 'MODOU', 'last': 'MATSU', 'n_life': 0, 'gametime': 0.0, 'family': 'time-1599447278_eve-3308179_name-MATSU', 'gen': 9}, {'player': 'de586e48a15f97f09e71355fcc82cea88d29633a', 'avatar': 3308595, 'tBirth': 1599462571, 'parent': 3308576, 'birth': '[-63788    148]', 'tDeath': 1599464786.0, 'death': '[-63794    187]', 'age': 36.92, 'cause_of_death': 'hunger', 'birthX': -63788, 'birthY': 148, 'deathX': -63794.0, 'deathY': 187.0, 'first': 'VIOLET', 'last': 'SUN', 'n_life': 36, 'gametime': 896.5699999999997, 'family': 'time-1599411200_eve-3307135_name-SUN', 'gen': 35}, {'player': '3da17539b16d6fe6911aefc57388bae9d1303e22', 'avatar': 3308591, 'tBirth': 15994623

Create new collection and save:

In [18]:
life_col = db.lifelogs
life_col.insert_many(life_records)

### Family collection

In [40]:
fam_in = fam_success.to_dict('records')
fam_records = []
for in_dict in notebook.tqdm(fam_in):
    fam = in_dict['family']
    members = life_df[life_df['family'] == fam]['avatar']
    members = list(members)
    
    out_dict = in_dict.copy()
    out_dict['members'] = members
    fam_records.append(out_dict)
    
print(fam_records[:5])


[{'family': 'time-1573253679_eve-2251043_name-MCCRACKEN', 'sum': 2, 'count': 9, 'a': 5, 'b': 10, 'beta_mean': 0.3333333333333333, 'beta_var': 0.013888888888888888, 'snr': 24.0, 'weighted_size': 3.0, 'longest_path': 3, 'members': [2251244, 2251239, 2251236, 2251234, 2251127, 2251125, 2251105, 2251079, 2251044, 2251043]}, {'family': 'time-1573253705_eve-2251049_name-FINCH', 'sum': 1, 'count': 8, 'a': 4, 'b': 10, 'beta_mean': 0.2857142857142857, 'beta_var': 0.013605442176870748, 'snr': 21.0, 'weighted_size': 2.2857142857142856, 'longest_path': 2, 'members': [2251267, 2251249, 2251223, 2251183, 2251161, 2251093, 2251050, 2251049]}, {'family': 'time-1573253707_eve-2251051_name-SORROWS', 'sum': 1, 'count': 3, 'a': 4, 'b': 5, 'beta_mean': 0.4444444444444444, 'beta_var': 0.02469135802469136, 'snr': 18.0, 'weighted_size': 1.3333333333333333, 'longest_path': 2, 'members': [2251139, 2251083, 2251051]}, {'family': 'time-1573253710_eve-2251053_name-LION', 'sum': 9, 'count': 40, 'a': 12, 'b': 34, '

Create new collection and save:

In [41]:
fam_col = db.families
fam_col.insert_many(fam_records)